In [3]:
import pandas as pd
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from pyecharts.charts import HeatMap
import pyecharts.options as opts
from copy import copy

In [4]:
from sklearn.model_selection import PredefinedSplit

In [67]:
from tqdm import tqdm

#### Since each model requires a lot of time to run, our team members run the code separately. Therefore, there is some code that is not running in this ipynb file.

### Data processing

In [5]:
data = pd.read_csv('/Users/zhouxiaomin/Desktop/hkust/COURSES NOTE/2021F/AI/Assignment 2/datashare/GKX_20201231.csv')

In [6]:
recent = data[~(data['DATE'] <= 19610101)]

In [13]:
recent

,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2,bm,bm_ia
358242,10006,19610131,5.396000e+04,0.322368,50.250000,1420,1.097917,1.205421,-0.131384,8.877138,...,0.017962,2.151394e-07,0.040134,0.016976,0.462846,0.708629,7.404632e-08,37.0,NaN,NaN
358243,10014,19610131,1.001600e+04,0.000000,4.000000,2504,1.528429,2.336095,-0.311111,6.895809,...,0.033141,9.621171e-07,0.093750,0.030237,0.854378,1.583680,6.630851e-08,39.0,NaN,NaN
358244,10030,19610131,6.243612e+04,0.087948,41.750000,1627,1.211479,1.467682,0.116933,8.932609,...,0.013038,3.012904e-07,0.029221,0.012935,0.580183,0.349018,1.538137e-07,NaN,NaN,NaN
358245,10057,19610131,2.362500e+04,0.142857,54.000000,500,1.070586,1.146155,0.034416,8.159303,...,0.013102,6.272476e-07,0.039548,0.014330,0.630725,0.580640,2.000000e+00,NaN,NaN,NaN
358246,10065,19610131,7.800000e+04,0.135417,27.250000,3250,0.849400,0.721481,0.077249,8.341708,...,0.010895,4.206203e-07,0.020942,0.010092,0.670171,0.273152,2.839977e-07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4345503,93423,20201231,2.611343e+06,0.109665,34.099998,84977,1.921340,3.691548,0.798194,16.060575,...,0.060029,4.462048e-10,0.176471,0.050187,0.519033,17.649093,3.803709e-09,79.0,NaN,NaN
4345504,93426,20201231,3.671460e+05,0.076239,31.480000,12552,1.301940,1.695047,0.551929,12.334190,...,0.035050,2.296462e-08,0.159100,0.043004,0.473367,2.144264,3.236729e-08,36.0,NaN,NaN
4345505,93427,20201231,2.523235e+06,0.135851,77.589996,36938,0.847169,0.717695,0.011311,14.568580,...,0.029388,1.364274e-09,0.048227,0.020206,0.294512,1.469369,2.227010e-08,36.0,NaN,NaN
4345506,93434,20201231,8.738802e+04,0.122605,2.930000,33482,0.389812,0.151954,0.282874,9.453983,...,0.051558,3.435272e-07,0.116667,0.035277,0.935967,0.897075,1.037670e-07,1.0,NaN,NaN


In [7]:
macro = pd.read_csv('macroeconomic predictors.csv')

In [8]:
macro

,Date,bm,tbl,ntis,svar,ep,dp,dfy,tms
0,196101,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
1,196102,0.512023,0.0242,0.019420,0.000547,-1.304053,-1.513817,0.0080,0.0150
2,196103,0.546680,0.0239,0.022738,0.000415,-1.323356,-1.525512,0.0080,0.0158
3,196104,0.545004,0.0229,0.023259,0.001009,-1.327841,-1.527178,0.0076,0.0162
4,196105,0.530916,0.0229,0.022466,0.000404,-1.338913,-1.535412,0.0074,0.0168
...,...,...,...,...,...,...,...,...,...
715,202008,0.235975,0.0010,-0.008504,0.000743,-1.550421,-1.772309,0.0102,0.0055
716,202009,0.241482,0.0011,-0.005698,0.004907,-1.534527,-1.756971,0.0105,0.0057
717,202010,0.253146,0.0010,-0.001895,0.003661,-1.528413,-1.746197,0.0109,0.0069
718,202011,0.226352,0.0009,-0.005262,0.002492,-1.578931,-1.791974,0.0100,0.0078


In [9]:
recent['Date'] = recent['DATE']/100

<ipython-input-9-071fb7a505ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent['Date'] = recent['DATE']/100


In [10]:
recent['Date'] = recent['Date'].astype(int)

<ipython-input-10-e41b323af78a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent['Date'] = recent['Date'].astype(int)


In [11]:
recent.set_index(['Date'],inplace=True)

In [12]:
macro.set_index(['Date'],inplace=True)

In [13]:
df = pd.merge(recent,macro,left_index=True,right_index=True,how='inner')

In [14]:
df

,permno,DATE,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,bm_x,bm_ia,bm_y,tbl,ntis,svar,ep_y,dp,dfy,tms
Date,,,,,,,,,,,,,,,,,,,,,
196101,10006,19610131,5.396000e+04,0.322368,50.250000,1420,1.097917,1.205421,-0.131384,8.877138,...,NaN,NaN,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
196101,10014,19610131,1.001600e+04,0.000000,4.000000,2504,1.528429,2.336095,-0.311111,6.895809,...,NaN,NaN,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
196101,10030,19610131,6.243612e+04,0.087948,41.750000,1627,1.211479,1.467682,0.116933,8.932609,...,NaN,NaN,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
196101,10057,19610131,2.362500e+04,0.142857,54.000000,500,1.070586,1.146155,0.034416,8.159303,...,NaN,NaN,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
196101,10065,19610131,7.800000e+04,0.135417,27.250000,3250,0.849400,0.721481,0.077249,8.341708,...,NaN,NaN,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202012,93423,20201231,2.611343e+06,0.109665,34.099998,84977,1.921340,3.691548,0.798194,16.060575,...,NaN,NaN,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
202012,93426,20201231,3.671460e+05,0.076239,31.480000,12552,1.301940,1.695047,0.551929,12.334190,...,NaN,NaN,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
202012,93427,20201231,2.523235e+06,0.135851,77.589996,36938,0.847169,0.717695,0.011311,14.568580,...,NaN,NaN,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084


In [15]:
df=df.fillna(0)

In [16]:
df = df[~df['RET'].isin([0])] 

In [17]:
df = df.drop('DATE',axis=1)

In [18]:
df = df.reset_index()

In [19]:
finaldata = df
finaldata

,Date,permno,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,bm_x,bm_ia,bm_y,tbl,ntis,svar,ep_y,dp,dfy,tms
0,196101,10006,5.396000e+04,0.322368,50.250000,1420,1.097917,1.205421,-0.131384,8.877138,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
1,196101,10030,6.243612e+04,0.087948,41.750000,1627,1.211479,1.467682,0.116933,8.932609,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
2,196101,10057,2.362500e+04,0.142857,54.000000,500,1.070586,1.146155,0.034416,8.159303,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
3,196101,10065,7.800000e+04,0.135417,27.250000,3250,0.849400,0.721481,0.077249,8.341708,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
4,196101,10102,3.053050e+05,0.032468,79.500000,3965,1.541293,2.375585,0.163292,10.033265,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3819001,202012,93423,2.611343e+06,0.109665,34.099998,84977,1.921340,3.691548,0.798194,16.060575,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
3819002,202012,93426,3.671460e+05,0.076239,31.480000,12552,1.301940,1.695047,0.551929,12.334190,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
3819003,202012,93427,2.523235e+06,0.135851,77.589996,36938,0.847169,0.717695,0.011311,14.568580,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
3819004,202012,93434,8.738802e+04,0.122605,2.930000,33482,0.389812,0.151954,0.282874,9.453983,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084


In [29]:
np.isnan(finaldata).any()

Date      False
permno    False
mvel1     False
RET       False
prc       False
          ...  
svar      False
ep_y      False
dp        False
dfy       False
tms       False
Length: 109, dtype: bool

### We define 2 functions here to facilitate programming.

#### 1. Out-of-sample R_square

In [30]:
def calculate_R_square(y_true, pre_ret):
    a = ((y_true-pre_ret)**2).sum()
    b = (y_true**2).sum()
    R_square = 1-(a/b)
    return R_square

#### 2. Split the data

In [31]:
def splitdata(data,i):
    train = data[data['Date'] < i]
    val = data.loc[train.shape[0]:]
    val = val[val['Date']<i+1200]   
    
    test = data.loc[(train.shape[0]+val.shape[0]):]
    test = test[test['Date'] < i+1300]
    
    #set x_train, y_train, x_val, y_val, x_test and y_test
    x_train=np.array(train.drop(['Date', 'RET','permno'],axis=1))
    y_train=np.array(train['RET']) 
    x_val=np.array(val.drop(['Date', 'RET','permno'],axis=1))
    y_val=np.array(val['RET'])
    x_test=np.array(test.drop(['Date', 'RET','permno'],axis=1))
    y_test=np.array(test['RET'])
    
    return x_train,y_train,x_val,y_val,x_test,y_test

## Lasso

In [93]:
R_square =[]

In [94]:
for i in range(197901,200901,100):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestLasso = None
                         
    for alpha in [0.8,1,10,100]:
        lasso = Lasso(alpha=alpha,normalize=True)
        lasso.fit(x_train,y_train)
    
        pred = lasso.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestLasso=lasso

        
    resultPred = bestLasso.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [95]:
R_square = pd.DataFrame(R_square)

In [96]:
R_square

,0
0,0.011994
1,0.007145
2,0.010987
3,-0.012353
4,0.015729
5,0.007060
6,0.008150
7,-0.004370
8,0.008466
9,-0.005266


In [97]:
R_square_lasso = R_square.mean()
R_square_lasso

0    0.003178
dtype: float64

## Ridge

In [98]:
R_square =[]

In [102]:
for i in tqdm(range(197901,200901,100)):

    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestRidge = None
                         
    for alpha in [0.1,0.4,1,10]:
        ridge = Ridge(alpha=alpha,normalize=True)
        ridge.fit(x_train,y_train)
    
        pred = ridge.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestRidge=ridge

        
    resultPred = bestRidge.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

100%|██████████| 30/30 [33:32<00:00, 67.08s/it] 


In [103]:
R_square = pd.DataFrame(R_square)

In [105]:
R_square

,0
0,0.034032
1,0.010807
2,0.008941
3,-0.017068
4,0.017542
5,0.004835
6,0.004903
7,-0.009168
8,0.002666
9,-0.005140


In [104]:
R_square_ridge = R_square.mean(axis=0)
R_square_ridge

0    0.005376
dtype: float64

## SDG

In [ ]:
R_square =[]

In [ ]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestsdg = None
                         
    for alpha in [0.01,0.1,0.001]:
        sdg = make_pipeline(StandardScaler(),SGDRegressor(loss='huber',alpha=alpha,max_iter=2000))
        sdg.fit(x_train,y_train)
    
        pred = sdg.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestsdg=sdg

        
    resultPred = bestsdg.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [ ]:
R_square = pd.DataFrame(R_square)

In [ ]:
R_square_sdg = R_square.mean(axis=0)
R_square_sdg

## PCA

In [106]:
R_square =[]

In [107]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestPca = None
    
                         
    for n_com in [10,15,20]:
        pca = PCA(n_components=n_com)
        pcr = make_pipeline(StandardScaler(), pca, LinearRegression())
        pcr.fit(x_train,y_train)
    
        pred = pcr.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestPca=pcr

        
    resultPred = bestPca.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

100%|██████████| 30/30 [1:10:29<00:00, 140.97s/it]


In [108]:
R_square = pd.DataFrame(R_square)

In [109]:
R_square 

,0
0,0.003875
1,-0.021820
2,-0.037682
3,-0.061603
4,-0.026957
5,-0.018863
6,0.004028
7,-0.006826
8,0.010037
9,-0.001675


In [110]:
R_square_pca = R_square.mean(axis=0)
R_square_pca

0   -0.005308
dtype: float64

## GBRT

In [111]:
R_square =[]

In [113]:
for i in tqdm(range(197901,200901,100)):
    
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestGbrt = None
                         
    for n_est in [10,15,20]:
        gbrt = GradientBoostingRegressor(n_estimators=n_est)
        gbrt.fit(x_train,y_train)
    
        pred = gbrt.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestGbrt=gbrt

        
    resultPred = bestGbrt.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

100%|██████████| 30/30 [34:41:17<00:00, 4162.57s/it]    


In [115]:
R_square = pd.DataFrame(R_square)

In [116]:
R_square 

,0
0,0.030183
1,0.021949
2,0.019002
3,-0.020418
4,0.020054
5,0.031947
6,0.034085
7,0.044596
8,0.031789
9,0.020800


In [117]:
R_square_gbrt = R_square.mean(axis=0)
R_square_gbrt

0    0.039834
dtype: float64

## Random Forest

In [16]:
R_square =[]    

In [ ]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestRf = None
                         
    for depth in [3,5]:
        for ntrees in [5,10]:
            rf = RandomForestRegressor(random_state=500, max_depth=depth, n_estimators=ntrees)
            rf.fit(x_train,y_train)
    
            pred = rf.predict(x_val)
            currentRSqr = calculate_R_square(y_val, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestRf=rf
                bestCombine=(depth,ntrees)
        
    resultPred = bestRf.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [ ]:
R_square = pd.DataFrame(R_square)

In [ ]:
R_square 

In [ ]:
R_square_rf = R_square.mean(axis=0)
R_square_rf

## Find top 1000 stocks by market equity 

In [26]:
marketv=finaldata[['permno','mvel1']]

In [27]:
mv=marketv.groupby('permno').mean().reset_index().sort_values('mvel1',ascending=False)

In [28]:
top=list(mv.head(1000).permno)
top

[14542,
 13407,
 13721,
 15488,
 16851,
 92611,
 92602,
 90319,
 89525,
 91233,
 15408,
 18576,
 86868,
 83443,
 84788,
 76076,
 87447,
 10107,
 10104,
 70519,
 66093,
 65875,
 12369,
 77178,
 89006,
 12308,
 66181,
 14008,
 18143,
 87031,
 92655,
 17778,
 13356,
 15054,
 14888,
 90215,
 16752,
 12558,
 86746,
 87842,
 17942,
 93436,
 55976,
 77418,
 59328,
 14593,
 59408,
 77274,
 86356,
 18298,
 90441,
 69032,
 88388,
 90505,
 89179,
 12972,
 13598,
 47896,
 89393,
 88668,
 18428,
 89428,
 13788,
 12345,
 60097,
 18163,
 11850,
 50876,
 43449,
 14541,
 59176,
 89258,
 11308,
 22111,
 22752,
 21936,
 13856,
 86783,
 26403,
 77702,
 87267,
 12490,
 91937,
 38703,
 17379,
 89954,
 19794,
 17782,
 91883,
 87055,
 18484,
 12622,
 12060,
 86580,
 22592,
 89071,
 18481,
 83835,
 19561,
 88239,
 82654,
 89626,
 13901,
 66157,
 86111,
 89199,
 19393,
 20482,
 14655,
 90857,
 83435,
 13286,
 89001,
 89444,
 12872,
 89130,
 88391,
 17830,
 91952,
 87212,
 86755,
 19286,
 93002,
 92156,
 66800,


In [29]:
data_top=pd.DataFrame()
for i in top:
    data_top = data_top.append(finaldata[finaldata['permno'].isin([i])],ignore_index=True)

In [30]:
data_top = data_top.sort_values(by="Date",ignore_index=True)

In [31]:
data_top

,Date,permno,mvel1,RET,prc,SHROUT,beta,betasq,chmom,dolvol,...,bm_x,bm_ia,bm_y,tbl,ntis,svar,ep_y,dp,dfy,tms
0,196101,18411,1.075296e+06,0.041667,50.000000,22402,0.365210,0.133378,-0.235507,10.483767,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
1,196101,10145,1.086195e+06,0.048276,57.000000,19976,1.299310,1.688207,0.003399,10.620043,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
2,196101,16678,3.920938e+05,0.004800,31.125000,12547,0.768105,0.589985,-0.129480,10.210444,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
3,196101,16600,2.754870e+05,0.093886,125.250000,2406,0.524370,0.274964,0.295407,9.551267,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
4,196101,24109,1.259422e+06,0.121739,64.500000,21903,0.483778,0.234041,-0.154714,10.608492,...,0.0,0.0,0.522987,0.0224,0.021815,0.000741,-1.284343,-1.501556,0.0078,0.0180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260021,202012,13618,2.188516e+07,0.028992,21.650000,1040169,0.898575,0.807436,0.326988,15.938613,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
260022,202012,13823,6.715761e+06,-0.052390,14.470000,439801,1.651530,2.727550,0.976780,15.668223,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
260023,202012,12542,5.772674e+06,0.290662,20.870001,356999,0.990907,0.981897,0.463906,16.273914,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084
260024,202012,12449,2.211904e+07,0.028732,272.829987,83402,1.074911,1.155433,0.293551,17.290835,...,0.0,0.0,0.219195,0.0009,-0.000094,0.000678,-1.601006,-1.809223,0.0090,0.0084


In [32]:
finaldata = data_top

## Lasso-top 1000 stocks

In [ ]:
R_square =[]

In [ ]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestLasso = None
                         
    for alpha in [0.8,1,10,100]:
        lasso = Lasso(alpha=alpha,normalize=True)
        lasso.fit(x_train,y_train)
    
        pred = lasso.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestLasso=lasso

        
    resultPred = bestLasso.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [ ]:
R_square = pd.DataFrame(R_square)

In [ ]:
R_square_lasso = R_square.mean(axis=0)
R_square_lasso

## Ridge- top 1000 stocks

In [ ]:
R_square =[]

In [ ]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestRidge = None
                         
    for alpha in [0.1,0.4,1,10]:
        ridge = Ridge(alpha=alpha,normalize=True)
        ridge.fit(x_train,y_train)
    
        pred = ridge.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestRidge=ridge

        
    resultPred = bestRidge.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [ ]:
R_square = pd.DataFrame(R_square)

In [ ]:
R_square_ridge = R_square.mean(axis=0)
R_square_ridge

## SDG-top 1000 stocks

In [ ]:
R_square =[]

In [ ]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestsdg = None
                         
    for alpha in [0.01,0.1,0.001]:
        sdg = make_pipeline(StandardScaler(),SGDRegressor(loss='huber',alpha=alpha,max_iter=2000))
        sdg.fit(x_train,y_train)
    
        pred = sdg.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestsdg=sdg

        
    resultPred = bestsdg.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [ ]:
R_square = pd.DataFrame(R_square)

In [ ]:
R_square_sdg = R_square.mean(axis=0)
R_square_sdg

## PCA-top 1000 stocks

In [ ]:
R_square =[]

In [ ]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestPca = None
    
                         
    for n_com in [10,15,20]:
        pca = PCA(n_components=n_com)
        pcr = make_pipeline(StandardScaler(), pca, LinearRegression())
        pcr.fit(x_train,y_train)
    
        pred = pcr.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestPca=pcr

        
    resultPred = bestPca.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [ ]:
R_square = pd.DataFrame(R_square)

In [ ]:
R_square_pca = R_square.mean(axis=0)
R_square_pca

## GBRT-top 1000 stocks

In [ ]:
R_square =[]

In [ ]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestGbrt = None
                         
    for n_est in [10,20,30]:
        gbrt = GradientBoostingRegressor(n_estimators=n_est)
        gbrt.fit(x_train,y_train)
    
        pred = gbrt.predict(x_val)
        currentRSqr = calculate_R_square(y_val, pred)
            
        if(currentRSqr>bestRSqr):
            bestRSqr=currentRSqr
            bestGbrt=gbrt

        
    resultPred = bestGbrt.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

In [ ]:
R_square = pd.DataFrame(R_square)

In [ ]:
R_square_gbrt = R_square.mean(axis=0)
R_square_gbrt

## Random Forest-top 1000 stocks

In [36]:
R_square =[]    

In [37]:
for i in tqdm(range(197901,200901,100)):
    
    x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
    bestRSqr = float("-Inf")
    bestRf = None
                         
    for depth in [3,5]:
        for ntrees in [5,10]:
            rf = RandomForestRegressor(random_state=500, max_depth=depth, n_estimators=ntrees)
            rf.fit(x_train,y_train)
    
            pred = rf.predict(x_val)
            currentRSqr = calculate_R_square(y_val, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestRf=rf
                bestCombine=(depth,ntrees)
        
    resultPred = bestRf.predict(x_test)
    resultRSqr = calculate_R_square(y_test, resultPred)

    R_square.append(resultRSqr)

100%|██████████| 30/30 [05:53<00:00, 11.77s/it]


In [38]:
R_square = pd.DataFrame(R_square)

In [39]:
R_square_rf = R_square.mean(axis=0)
R_square_rf

0    0.020917
dtype: float64

# Feature importance

### Variable Selection by correlation 

In [33]:
final = finaldata.iloc[:,:101]

In [34]:
da = final.drop(['Date','permno','RET','mom1m', 'chmom', 'indmom','mom12m','maxret','dolvol','dy','securedind','nincr','retvol','mvel1','std_turn','agr'],axis = 1)

In [35]:
class cal_corr:
    
    def __init__(self, df, n_clusters=5, threshold=0.01):
        self.df = self.handle_df(df.copy())
        self.n_clusters = n_clusters
        self.threshold = threshold
        self.corr = None
        
    @property
    def corr_matrix(self):
        return self.corr
    
    @property
    def corr_pairs(self):
        return self.pairs
        
        
    def handle_df(self,df):
        for item in df.select_dtypes(include=["object"]):
            df[item] = df[item].astype('category').cat.codes
        
        df = pd.DataFrame(StandardScaler().fit_transform(df),columns=df.columns)
        print(df.shape)
        return df
        
    def order_by_kmeans(self):
        kk = KMeans(n_clusters = self.n_clusters)
        res = kk.fit_predict(self.df.T)
        
        self.df = self.df.append(pd.Series(res,index=self.df.columns),ignore_index=True)
        self.df.sort_values(by = self.df.shape[0]-1,axis=1,inplace=True)
        self.df.drop(self.df.shape[0]-1,inplace=True)
        
        print(self.df.shape)
        
    def corr_heat_map(self):
        self.order_by_kmeans()
        
        self.corr = self.df.corr(method="pearson")
        self.corr = self.corr.round(3)
        self.corr = self.corr.apply(lambda x: abs(x))
        myvalues = []
        for i in range(len(self.corr.index)):
            for j in range(len(self.corr.index)):
                tmp = [i, j, self.corr.iloc[i, j]]
                myvalues.append(copy(tmp))
        self.__setattr__('pairs',myvalues)        
        
        heat_map = HeatMap(init_opts=opts.InitOpts(width="1440px", height="1440px"))\
            .add_xaxis(list(self.corr.columns))\
            .add_yaxis("corr", list(self.corr.index), myvalues) \
            .set_global_opts(
                title_opts=opts.TitleOpts(title="Correlation of model variables"),
                datazoom_opts=[opts.DataZoomOpts(is_show=True, is_realtime=True), ],
                visualmap_opts=opts.VisualMapOpts(min_=-1.2, max_=1.2, pos_right=20),
                toolbox_opts=opts.ToolboxOpts(is_show=True),
                xaxis_opts=opts.AxisOpts(type_="category", is_scale=True, is_inverse=True, axislabel_opts=opts.LabelOpts(is_show=True, rotate=-60)),
                yaxis_opts=opts.AxisOpts(is_scale=True, is_inverse=False, axislabel_opts=opts.LabelOpts(is_show=True, position="right")),
                tooltip_opts=opts.TooltipOpts(is_show=True))\
            .set_series_opts(label_opts=opts.LabelOpts(is_show=True, position="insideBottom"))

        heat_map.render()
        

    def drop_hight_corr(self):
        cor_pair = self.pairs
        cor_pair.sort(key=lambda x:x[2],reverse=True)
        del_pair = []
        del_col = []
        for item in cor_pair:
            if (item[0]==item[1])|(set(item[:2]) in del_pair)|(item[0] in del_col)|(item[1] in del_col):
                continue

            if item[2]>self.threshold:
                c1 = self.corr.iloc[item[0],[x for x in range(self.corr.shape[0]) if x not in del_col]].mean()
                c2 = self.corr.iloc[item[1],[x for x in range(self.corr.shape[0]) if x not in del_col]].mean()

                del_col.append(item[0] if c1>c2 else item[1])
                del_pair.append(set(item[:2]))
            else:
                break
            
        del_col_name = self.corr.iloc[:,del_col].columns
        
        return list(del_col_name)


if __name__ == '__main__':
    c = cal_corr(da, n_clusters=5, threshold=0.01)
    c.corr_heat_map()
    del_col = c.drop_hight_corr()


(260026, 85)
(260026, 85)


In [23]:
var = finaldata.columns.values.tolist()
for i in del_col:
    var.remove(i)

In [24]:
var.remove('Date')
var.remove('permno')
var.remove('RET')

### Lasso

In [64]:
R_square_lasso_=pd.DataFrame()

In [69]:
for j in tqdm(var):
    finaldata = data_top.copy()
    finaldata.j=0
    R_square =[]
    for i in range(197901,200901,100):
        
        x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
        bestRSqr = float("-Inf")
    
                         
        for alpha in [0.8,1,10,100]:
            lasso = Lasso(alpha=alpha,normalize=True)
            lasso.fit(x_train,y_train)
    
            pred = lasso.predict(x_val)
            currentRSqr = calculate_R_square(y_val, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestLasso=lasso

        
        resultPred = bestLasso.predict(x_test)
        resultRSqr = calculate_R_square(y_test, resultPred)

        R_square.append(resultRSqr)
    R_square_lasso_ = R_square_lasso_.append(pd.DataFrame(R_square).mean(axis=0), ignore_index=True)
        

100%|██████████| 32/32 [02:56<00:00,  5.51s/it]


In [84]:
Lasso=pd.DataFrame({'Predictors':var}).merge(R_square_lasso_,left_index=True,right_index=True)

In [85]:
Lasso.to_csv('R_square_lasso.csv',index=False)

### Ridge

In [ ]:
R_square_ridge_=pd.DataFrame()

In [ ]:
for j in tqdm(var):
    finaldata = data_top.copy()
    finaldata.j=0
    R_square =[]
    for i in range(197901,200901,100):
        
        x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
        bestRSqr = float("-Inf")
                         
        for alpha in [0.1,0.4,1,10]:
            ridge = Ridge(alpha=alpha,normalize=True)
            ridge.fit(x_train,y_train)
    
            pred = ridge.predict(x_val)
            currentRSqr = calculate_R_square(y_val, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestRidge=ridge

        
        resultPred = bestRidge.predict(x_test)
        resultRSqr = calculate_R_square(y_test, resultPred)

        R_square.append(resultRSqr)
    R_square_ridge_ = R_square_ridge_.append(pd.DataFrame(R_square).mean(axis=0), ignore_index=True)
        

In [ ]:
Ridge=pd.DataFrame({'Predictors':var}).merge(R_square_ridge_,left_index=True,right_index=True)

In [ ]:
Ridge.to_csv('R_square_ridge.csv',index=False)

### SDG

In [ ]:
R_square_sdg_=pd.DataFrame()

In [ ]:
for j in tqdm(var):
    finaldata = data_top.copy()
    finaldata.j=0
    R_square =[]
    for i in range(197901,200901,100):
        
        x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
        bestRSqr = float("-Inf")
                         
        for alpha in [0.01,0.1,0.001]:
            sdg = make_pipeline(StandardScaler(), pca, LinearRegression())
            sdg.fit(x_train,y_train)
    
            pred = sdg.predict(x_val)
            currentRSqr = calculate_R_square(y_val, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestsdg=sdg

        
        resultPred = bestsdg.predict(x_test)
        resultRSqr = calculate_R_square(y_test, resultPred)

        R_square.append(resultRSqr)
    R_square_sdg_ = R_square_sdg_.append(pd.DataFrame(R_square).mean(axis=0), ignore_index=True)
        

In [ ]:
R_square_sdg_.to_csv('R_square_sdg.csv')

### PCA

In [ ]:
R_square_pca_=pd.DataFrame()

In [ ]:
for j in tqdm(var):
    finaldata = data_top.copy()
    finaldata.j=0
    R_square =[]
    for i in range(197901,200901,100):
        
        x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
        bestRSqr = float("-Inf")

    
                         
        for n_com in [10,15,20]:
            pca = PCA(n_components=n_com)
            pcr = make_pipeline(StandardScaler(), pca, LinearRegression())
            pcr.fit(x_train,y_train)
    
            pred = pcr.predict(x_val)
            currentRSqr = calculate_R_square(y_val, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestPca=pcr

        
        resultPred = bestPca.predict(x_test)
        resultRSqr = calculate_R_square(y_test, resultPred)

        R_square.append(resultRSqr)
    R_square_pca_ = R_square_pca_.append(pd.DataFrame(R_square).mean(axis=0), ignore_index=True)

In [ ]:
R_square_pca_.to_csv('R_square_pca.csv')

### GBRT

In [ ]:
R_square_gbrt_=pd.DataFrame()

In [ ]:
for j in tqdm(var):
    finaldata = data_top.copy()
    finaldata.j=0
    R_square =[]
    for i in range(197901,200901,100):
        
        x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
        bestRSqr = float("-Inf")
   
                         
        for n_est in [10,20,30]:
            gbrt = GradientBoostingRegressor(n_estimators=n_est)
            gbrt.fit(x_train,y_train)
    
            pred = gbrt.predict(x_val)
            currentRSqr = calculate_R_square(y_val, pred)
            
            if(currentRSqr>bestRSqr):
                bestRSqr=currentRSqr
                bestGbrt=gbrt

        
        resultPred = bestGbrt.predict(x_test)
        resultRSqr = calculate_R_square(y_test, resultPred)

        R_square.append(resultRSqr)
    R_square_gbrt_ = R_square_gbrt_.append(pd.DataFrame(R_square).mean(axis=0), ignore_index=True)

In [ ]:
R_square_gbrt_.to_csv('R_square_gbrt.csv')

### Random Forest

In [40]:
R_square_rf_=pd.DataFrame()

In [41]:
for j in tqdm(var):
    finaldata = data_top.copy()
    finaldata.j=0
    R_square =[]
    for i in range(197901,200901,100):
        
        x_train,y_train,x_val,y_val,x_test,y_test=splitdata(finaldata,i)
    
        bestRSqr = float("-Inf")

                         
        for depth in [3,5]:
            for ntrees in [5,10]:
                rf = RandomForestRegressor(random_state=500, max_depth=depth, n_estimators=ntrees)
                rf.fit(x_train,y_train)
    
                pred = rf.predict(x_val)
                currentRSqr = calculate_R_square(y_val, pred)
            
                if(currentRSqr>bestRSqr):
                    bestRSqr=currentRSqr
                    bestRf=rf
                    bestCombine=(depth,ntrees)
        
        resultPred = bestRf.predict(x_test)
        resultRSqr = calculate_R_square(y_test, resultPred)

        R_square.append(resultRSqr)
    R_square_rf_ = R_square_rf_.append(pd.DataFrame(R_square).mean(axis=0), ignore_index=True)

100%|██████████| 32/32 [11:08:00<00:00, 1252.53s/it]  


In [43]:
R_square_rf_.to_csv('R_square_rf.csv')